In [ ]:
%reload_ext autoreload
%autoreload 2
    
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
matplotlib.rcParams['svg.fonttype'] = 'none' # for svgs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import seaborn as sns
import pickle
from tqdm import tqdm
from scipy import stats
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression

import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_map.figure_utils import depth_selectivity, rf, get_session_list, roi_location
from v1_depth_map.figure_utils import common_utils as plt_common_utils
from cottage_analysis.analysis.common_utils import ztest_2d

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
READ_VERSION = 10
VERSION = 11
READ_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_figures"/f"ver{READ_VERSION}"
SAVE_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_figures"/f"ver{VERSION}"
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
reload=False

In [ ]:
# Load data for example RF
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
session_name_rf = "PZAH8.2h_S20230116"
vs_df_rf, trials_df_rf = spheres.sync_all_recordings(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)
frames_rf, imaging_df_rf = spheres.regenerate_frames_all_recordings(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
    resolution=1,
)
neurons_ds_rf = pipeline_utils.create_neurons_ds(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    project=project,
    conflicts="skip",
)
neurons_df_rf = pd.read_pickle(neurons_ds_rf.path_full)

In [ ]:
# Load data for calculating proportion of significant RF
if reload:
    project = "hey2_3d-vision_foodres_20220101"
    flexilims_session = flz.get_flexilims_session(project)
    all_sig, all_sig_ipsi, neurons_df_all = rf.load_sig_rf(
        flexilims_session=flexilims_session,
        session_list=get_session_list.get_sessions(
            flexilims_session=flexilims_session,
            exclude_openloop=False,
            exclude_pure_closedloop=False,
            v1_only=True,
        ),
        n_std=6,
    )

    neurons_df_all = roi_location.align_across_mice(neurons_df_all) # align neuron coordinates across mice
    neurons_df_all.to_pickle(SAVE_ROOT / "rf_neurons_df_all.pkl")

    with open(SAVE_ROOT / "rf_all_sig.pkl", "wb") as f:
        pickle.dump(all_sig, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(SAVE_ROOT / "rf_all_sig_ipsi.pkl", "wb") as f:
        pickle.dump(all_sig_ipsi, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
    neurons_df_all = pd.read_pickle(READ_ROOT / "rf_neurons_df_all.pkl")
    all_sig = pickle.load(open(READ_ROOT / "rf_all_sig.pkl", "rb"))
    all_sig_ipsi = pickle.load(open(READ_ROOT / "rf_all_sig_ipsi.pkl", "rb"))

## 3D RFs


In [ ]:
fontsize_dict = {"title": 7, "label": 7, "tick": 6, "legend": 5}
fig = plt.figure(figsize=(18 / 2.54, 18 / 2.54))

# Example reconstructed stimuli frame
trial_idx = 4
example_trial = trials_df_rf.iloc[trial_idx]
depths = np.sort(trials_df_rf.depth.unique())
idepth = np.nonzero(depths == example_trial.depth)[0][0]
iframe = (trials_df_rf.iloc[trial_idx].imaging_stim_start + 10,)
rf.plot_stimulus_frame(
    frame=frames_rf[iframe, :, frames_rf.shape[2] // 2 :].squeeze(),
    idepth=idepth,
    depths=depths,
    position=[0, 0.74, 0.3, 0.4],
    fontsize_dict=fontsize_dict,
    plot_prop=0.9,
)

# Example receptive field of 3 V1 neurons
SELECT_ROIS = [
    319,
    720,
    120,
]
for iroi, (roi, linecolor) in enumerate(zip(SELECT_ROIS, ["red", "green", "blue"])):
    fig.add_axes([0.267 + 0.165 * iroi, 0.9, 0.05, 0.05])
    depth_tuning_kwargs = dict(
        rs_thr=None,
        plot_fit=True,
        plot_smooth=False,
        linewidth=1,
        closed_loop=1,
        fontsize_dict=fontsize_dict,
        markersize=5,
        markeredgecolor='w',
    )
    depth_selectivity.plot_depth_tuning_curve(
        neurons_df=neurons_df_rf,
        trials_df=trials_df_rf,
        roi=roi,
        linecolor=linecolor,
        ylim_precision_base=5,
        ylim_precision=2,
        **depth_tuning_kwargs,
    )
    plt.ylabel("")
    plt.xlabel("")
    plt.gca().set_xticklabels([])
    plt.gca().tick_params(length=1.5)
    if iroi == len(SELECT_ROIS) // 2:
        xlabel = "Azimuth (degrees)"
    else:
        xlabel = ""
    if iroi == 0:
        ylabel = "Elevation (degrees)"
    else:
        ylabel = ""
    rf.plot_rf(
        neurons_df=neurons_df_rf,
        roi=roi,
        ndepths=8,
        frame_shape=(16, 24),
        position=[0.055 + 0.165 * iroi, 0.828, 0.5, 0.5],
        plot_prop=0.9,
        xlabel=xlabel,
        ylabel=ylabel,
        fontsize_dict=fontsize_dict,
    )
plt.savefig(SAVE_ROOT / "fig_rf_examples.pdf", bbox_inches="tight")

In [ ]:
# Plot 3D RFs
rf.plot_rf_3d(
    neurons_df_rf,
    SELECT_ROIS,
    depths,
    SAVE_ROOT / "fig_3d_rfs.pdf",
    dict(label=10, tick=10),
)

In [ ]:
# Proportion of neurons with significant receptive fields across imaging sessions
depth_selective = ((neurons_df_all["iscell"] == 1)
    & (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (neurons_df_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1))
rf_sig = (neurons_df_all["rf_sig"] & depth_selective)
print(f"{np.sum(rf_sig)} RF sig neurons out of {np.sum(depth_selective)} depth selective neurons, proportion {np.sum(rf_sig)/np.sum(depth_selective)}")

depths = np.sort(trials_df_rf.depth.unique())
fontsize_dict = {"title": 7, "label": 7, "tick": 6, "legend": 5}
fig = plt.figure(figsize=(18 / 2.54, 18 / 2.54))
fig.add_axes([0.1, 0.2, 0.2, 0.2])
rf.plot_sig_rf_perc(
    all_sig=all_sig,
    all_sig_ipsi=all_sig_ipsi,
    plot_type="hist",
    hist_color="cornflowerblue",
    hist_edgecolor="royalblue",
    bins=np.arange(0, 1, 0.1),
    fontsize_dict=fontsize_dict,
)
print(f"ipsi side significant RF {np.median(all_sig_ipsi)}")

# Plot example session, spatial distribution of preferred azimuth/elevation/depth
suite2p_ds = flz.get_datasets(
    flexilims_session=flexilims_session,
    origin_name=session_name_rf,
    dataset_type="suite2p_rois",
    filter_datasets={"anatomical_only": 3},
    allow_multiple=False,
    return_dataseries=False,
)
iscell = np.load(suite2p_ds.path_full / "plane0" / "iscell.npy", allow_pickle=True)[
    :, 0
]
neurons_df_rf["iscell"] = iscell
stat = np.load(suite2p_ds.path_full / "plane0" / "stat.npy", allow_pickle=True)
ops = np.load(suite2p_ds.path_full / "plane0" / "ops.npy", allow_pickle=True).item()

# azimuth
fig.add_axes([0.07, 0.5, 0.25, 0.25])
im = depth_selectivity.plot_example_fov(
    neurons_df=neurons_df_rf,
    ops=ops,
    stat=stat,
    ndepths=len(depths),
    col="rf_azi",
    cmap=cm.YlOrRd.reversed(),
    background_color=np.array([0, 0, 0]),
    fontsize_dict=fontsize_dict,
    fov_width=661,
)

# elevation
fig.add_axes([0.37, 0.5, 0.25, 0.25])
im = depth_selectivity.plot_example_fov(
    neurons_df=neurons_df_rf,
    ops=ops,
    stat=stat,
    ndepths=len(depths),
    col="rf_ele",
    cmap=cm.YlOrRd.reversed(),
    background_color=np.array([0, 0, 0]),
    fontsize_dict=fontsize_dict,
    fov_width=661,
)

# preferred depth
fig.add_axes([0.65, 0.5, 0.25, 0.25])
im = depth_selectivity.plot_example_fov(
    neurons_df=neurons_df_rf,
    ops=ops,
    stat=stat,
    ndepths=len(depths),
    col="preferred_depth_closedloop",
    cmap=cm.cool.reversed(),
    background_color=np.array([0, 0, 0]),
    fontsize_dict=fontsize_dict,
    fov_width=661,
)
plt_common_utils.plot_white_rectangle(0.805, 0.685, 0.2, 0.2)
fig.add_axes([0.81, 0.68, 0.1, 0.1])
depth_selectivity.plot_fov_mean_img(ops["meanImg"], fov_width=661,)

plt.savefig(SAVE_ROOT / "fig_rf_example_fov.svg", dpi=300)

## Preferred depth map


In [ ]:
# Calculate the smoothed map of preferred azimuth / elevation / depth across V1
overview_img = roi_location.check_neurons_in_v1(neurons_df_all,
                                                v1_mask_fname= flz.get_data_root("processed", flexilims_session=flexilims_session) / project / "PZAH10.2d/FOVs/V1_mask_2.roi",
                                                overview_fname=flz.get_data_root("processed", flexilims_session=flexilims_session) / project / "PZAH10.2d/FOVs/PZAH10.2d_overview.tif",)
neurons_df_all["preferred_depth_corrected"] = neurons_df_all[
    "preferred_depth_closedloop"
] / np.sqrt(
    (np.sin(np.deg2rad(neurons_df_all["rf_azi"])) ** 2)
    * (np.cos(np.deg2rad(neurons_df_all["rf_ele"])) ** 2)
    + (np.sin(np.deg2rad(neurons_df_all["rf_ele"])) ** 2)
)

sig_neurons = (
    (neurons_df_all["iscell"] == True)
    & (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (neurons_df_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (neurons_df_all["v1"] == True)
    & (neurons_df_all["rf_sig"] == True)
)
neurons_df_all["log_preferred_depth"] = neurons_df_all[
    "preferred_depth_closedloop"
].apply(lambda x: np.log(x))
neurons_df_all["log_preferred_depth_corrected"] = neurons_df_all[
    "preferred_depth_corrected"
].apply(lambda x: np.log(x))
xrange = int(neurons_df_all["overview_x_aligned"].min()), int(
    neurons_df_all["overview_x_aligned"].max()
)
yrange = int(neurons_df_all["overview_y_aligned"].min()), int(
    neurons_df_all["overview_y_aligned"].max()
)
xx, yy = np.mgrid[xrange[0] : xrange[1] : 10, yrange[0] : yrange[1] : 10]
maps = {}
maps_alpha = {}
for icol, col in enumerate(
    ["rf_azi", "rf_ele", "log_preferred_depth", "log_preferred_depth_corrected"]
):
    # for every xx and yy, calculated the spatially weighted sum of the neurons as a function of distance
    zz = np.ravel(np.zeros(xx.shape))
    alpha = np.ravel(np.zeros(xx.shape))
    length_scale = 25
    pixel_size = 6600/(2064/2) #widefield camera: 6.6 mm for 1032 pixels (2064 pixels binned 2x2)
    print(f"Gaussian kernel length scale {length_scale/np.sqrt(2)*pixel_size} um")
    for i, (x, y) in tqdm(enumerate(zip(np.ravel(xx), np.ravel(yy))), total=xx.size):
        weights = np.exp(
            -(
                (neurons_df_all[sig_neurons]["overview_x_aligned"] - x) ** 2
                + (neurons_df_all[sig_neurons]["overview_y_aligned"] - y) ** 2
            )
            / length_scale**2
        )
        alpha[i] = np.sum(weights)
        zz[i] = (
            np.dot(
                weights,
                neurons_df_all[sig_neurons][col].values,
            )
            / alpha[i]
        )
    maps[col] = np.reshape(zz, xx.shape).T
    maps_alpha[col] = np.reshape(alpha, xx.shape).T

In [ ]:
# Plot the map of preferred azimuth / elevation / depth across V1
fig = plt.figure(figsize=(18 / 2.54, 7 / 2.54))
ax = fig.add_axes([0.05, 0.2, 0.65, 0.65])
plt.imshow(overview_img)
plt.xticks([])
plt.yticks([])
xlims = [50, overview_img.shape[1] - 50]
ylims = [50, overview_img.shape[0] - 50]
plt.xlim(xlims)
plt.ylim(ylims)
plt.gca().invert_yaxis()

scatter_alpha=0.3
# Add scalebar
scalebar_length_px = 1000/pixel_size  # Scale bar of 1mm
rect = plt.Rectangle(
    (overview_img.shape[1] - 50 - 180, 
     overview_img.shape[0] - 50 - 30), 
    scalebar_length_px, 
    scalebar_length_px*0.05, 
    color="white"
)
plt.gca().add_patch(rect)
for icol, col in enumerate(["rf_azi", "rf_ele", "log_preferred_depth"]):
    ax = fig.add_axes([0.39 + 0.25 * icol, 0.55, 0.35, 0.35])
    if col in ["log_preferred_depth_corrected", "log_preferred_depth"]:
        palette = cm.cool.reversed()
        vrange = [np.log(0.1), np.log(2)]
    else:
        palette = cm.YlOrRd.reversed()
        vrange = (
            neurons_df_all[sig_neurons][col].min(),
            neurons_df_all[sig_neurons][col].max(),
        )

    # clip data to vrange
    neurons_df_all_cp = neurons_df_all.copy()
    neurons_df_all_cp.loc[(neurons_df_all_cp[col] < vrange[0]), col] = vrange[0]
    neurons_df_all_cp.loc[(neurons_df_all_cp[col] > vrange[1]), col] = vrange[1]
    if col in ["log_preferred_depth_corrected", "log_preferred_depth"]:
        print(f"data range {col}: {np.exp(neurons_df_all_cp[col].min())} - {np.exp(neurons_df_all_cp[col].max())}")
    else:
        print(f"data range {col}: {neurons_df_all_cp[col].min()} - {neurons_df_all_cp[col].max()}")
    ax = sns.scatterplot(
        neurons_df_all_cp[sig_neurons],
        x="overview_x_aligned",
        y="overview_y_aligned",
        hue=col,
        palette=palette,
        s=1,
        alpha=scatter_alpha,
        linewidth=0,
        legend=False,
        ax=ax,
        vmax=vrange[1],
        vmin=vrange[0],
        rasterized=True,
    )
    norm = plt.Normalize(vmin=vrange[0], vmax=vrange[1])
    sm = plt.cm.ScalarMappable(cmap=palette, norm=norm)
    sm.set_array([])
    ax.set_aspect("equal", adjustable="box")
    plt.xlabel("")
    plt.ylabel("")
    plt.xlim(xlims)
    plt.ylim(ylims)
    ax.invert_yaxis()
    ax.invert_xaxis()
    plt.xticks([])
    plt.yticks([])

    alpha = maps_alpha[col]
    max_alpha = alpha.max()
    alpha = alpha / max_alpha * 5
    alpha = np.clip(alpha, 0, 1)
    ax = fig.add_axes([0.39 + 0.25 * icol, 0.15, 0.35, 0.35])
    im = ax.imshow(
        maps[col],
        extent=(xrange[0], xrange[1], yrange[0], yrange[1]),
        origin="lower",
        alpha=alpha,
        cmap=palette,
        vmin=vrange[0],
        vmax=vrange[1],
    )
    if col in ["log_preferred_depth_corrected", "log_preferred_depth"]:
        print(f"map range {col}: {np.exp(maps[col].min())} - {np.exp(maps[col].max())}")
    else:
        print(f"map range {col}: {maps[col].min()} - {maps[col].max()}")
    plt.xlim(xlims)
    plt.ylim(ylims)
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.xticks([])
    plt.yticks([])

    plot_x, plot_y, plot_width, plot_height = ax.get_position().x0, ax.get_position().y0, ax.get_position().width, ax.get_position().height
    ax2 = fig.add_axes(
        [plot_x + plot_width * 1.1, plot_y, plot_width*0.1, plot_height / 2]
    )
    cbar=plt.colorbar(mappable=sm, cax=ax2)
    ax2.tick_params(labelsize=fontsize_dict["legend"], length=2, pad=2)
    ax2.set_ylabel(
        "\u0394F/F", rotation=270, fontsize=fontsize_dict["legend"], labelpad=4
    )
    if col in ["log_preferred_depth_corrected", "log_preferred_depth"]:
        cbar.set_ticks(vrange)
        cbar.ax.set_yticklabels([f"<{np.around(np.exp(vrange[0])*100).astype('int')}", 
                                 f">{np.around(np.exp(vrange[1])*100).astype('int')}"])
    else:
        cbar.set_ticks(vrange)
        cbar.ax.set_yticklabels([f"{np.around(vrange[0],1)}",
                                 f"{np.around(vrange[1],1)}"])
    cbar.ax.tick_params(labelsize=fontsize_dict["tick"])

plt.savefig(SAVE_ROOT / f"v1_map_vrange_{np.round(np.exp(vrange[0]),1)}_{np.round(np.exp(vrange[1]))}_a{scatter_alpha}.svg", bbox_inches="tight", dpi=300)

for colx in ["overview_x_aligned", "overview_y_aligned"]:
    for coly in ["log_preferred_depth", "log_preferred_depth_corrected"]:
        print(f"Correlation R {colx} vs {coly}: {spearmanr(neurons_df_all[sig_neurons][colx], neurons_df_all[sig_neurons][coly])[0]}")


In [ ]:
# Calculate the gradient of preferred depth with respect to azimuth and elevation
neurons_df_all["log_preferred_depth_corrected"] = neurons_df_all[
    "preferred_depth_corrected"
].apply(lambda x: np.log(x))

n_boot = 20000
if reload:
    corrs = {
        "rf_azi": [],
        "rf_ele": [],
        "z_position": [],
        "overview_x_aligned": [],
        "overview_y_aligned": [],
    }
    np.random.seed(0)
    coefs = []
    coefs_corrected = []
    for i in tqdm(range(n_boot)):
        sample = common_utils.bootstrap_sample(
            neurons_df_all[sig_neurons], ["mouse", "session"]
        )
        for col in corrs:
            # corrs[col].append(
            #     spearmanr(
            #         neurons_df_all.loc[sample, col],
            #         neurons_df_all.loc[sample, "preferred_depth_closedloop"],
            #     )[0]
            # )
            corrs[col].append(
                spearmanr(
                    neurons_df_all.loc[sample, col],
                    neurons_df_all.loc[sample, "log_preferred_depth"],
                )[0]
            )
        # determine how col depends on rf_azi and rf_ele
        X = neurons_df_all.loc[sample, ["rf_azi", "rf_ele"]].values
        y = neurons_df_all.loc[sample, "log_preferred_depth"].values
        reg = LinearRegression().fit(X, y)
        coefs.append(reg.coef_)

        y = neurons_df_all.loc[sample, "log_preferred_depth_corrected"].values
        reg = LinearRegression().fit(X, y)
        coefs_corrected.append(reg.coef_)
        
    rf_gradient_bootstraps = {
        "coefs": coefs,
        "coefs_corrected": coefs_corrected,
        "corrs": corrs,
    }
    with open(SAVE_ROOT / "rf_gradient_bootstraps.pkl", "wb") as f:
        pickle.dump(rf_gradient_bootstraps, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(READ_ROOT / "rf_gradient_bootstraps.pkl", "rb") as f:
        rf_gradient_bootstraps = pickle.load(f)
    coefs = np.array(rf_gradient_bootstraps["coefs"])
    coefs_corrected = np.array(rf_gradient_bootstraps["coefs_corrected"])
    corrs = rf_gradient_bootstraps["corrs"]

In [ ]:
# Plot the gradient of preferred depth with respect to azimuth and elevation
mode = "corrected" # mode can be "corrected" or "uncorrected"
coefs_corrected = np.array(coefs_corrected)
coefs = np.array(coefs)
if mode == "corrected":
    this_col = "preferred_depth_corrected"
    this_coef = coefs_corrected
elif mode == "uncorrected":
    this_col = "preferred_depth_closedloop"
    this_coef = coefs
fig = plt.figure(figsize=(18 / 2.54, 10 / 2.54))
fontsize_dict = {"title": 7, "label": 7, "tick": 6, "legend": 5}

neurons_df_all["rf_azi_jittered"] = neurons_df_all["rf_azi"] + np.random.uniform(
    -2, 2, neurons_df_all.shape[0]
)
neurons_df_all["rf_ele_jittered"] = neurons_df_all["rf_ele"] + np.random.uniform(
    -2, 2, neurons_df_all.shape[0]
)

neurons_df_sig = neurons_df_all[sig_neurons]
xrange = 17.5, 117.5
yrange = -37.5, 37.5
xx, yy = np.mgrid[xrange[0] : xrange[1] + 5 : 5, yrange[0] : yrange[1] + 5 : 5]

# for every xx and yy, calculated the spatially weighted sum of the neurons as a function of distance
ranges = [[-np.inf, 0.2], [0.2, 1], [1, np.inf]]

zz_near = np.ravel(np.zeros(xx.shape))
zz_mid = np.ravel(np.zeros(xx.shape))
zz_far = np.ravel(np.zeros(xx.shape))
length_scale = 5
col = this_col


def format_plot(ax):
    sns.despine(offset=2, ax=ax)
    ax.set_xticks([30, 60, 90, 120])
    ax.set_yticks([-40, 0, 40])
    ax.tick_params(labelsize=fontsize_dict["tick"], length=2, direction="out")


def calculate_zz(condition):
    zz = np.ravel(np.zeros(xx.shape))
    for i, (x, y) in tqdm(enumerate(zip(np.ravel(xx), np.ravel(yy))), total=xx.size):
        idx = (neurons_df_sig["rf_azi"] == x) & (neurons_df_sig["rf_ele"] == y)
        if np.sum(idx) < 20:
            zz[i] = np.nan
        else:
            zz[i] = np.mean(condition[idx])
    return np.reshape(zz, xx.shape).T


im_extent = (xrange[0] - 2.5, xrange[1] + 2.5, yrange[0] - 2.5, yrange[1] + 2.5)

median_depth = np.ravel(np.zeros(xx.shape))
for i, (x, y) in tqdm(enumerate(zip(np.ravel(xx), np.ravel(yy))), total=xx.size):
    idx = (neurons_df_sig["rf_azi"] == x) & (neurons_df_sig["rf_ele"] == y)
    if np.sum(idx) < 20:
        median_depth[i] = np.nan
    else:
        median_depth[i] = np.median(neurons_df_sig[idx][col])
median_depth = np.reshape(median_depth, xx.shape).T

vrange = [np.log(0.05), np.log(6.4)]

# Plot median preferred depth binned by azimuth and elevation
ax = fig.add_axes([0.05, 0.1, 0.35, 0.35])
im = ax.imshow(
    np.log(median_depth),
    extent=im_extent,
    origin="lower",
    cmap="cool_r",
    vmin=vrange[0],
    vmax=vrange[1],
)
print(f"vrange for colormap: {np.exp(vrange)}")
plt.xlabel("RF azimuth (degrees)", fontsize=fontsize_dict["label"])
plt.ylabel("RF elevation (degrees)", fontsize=fontsize_dict["label"])
format_plot(ax)
sns.despine()

ax_pos = ax.get_position()
cbar_ax = plt.gcf().add_axes(
    [
        ax_pos.x0 + ax_pos.width * 1.05,
        ax_pos.y0,
        0.01,
        ax_pos.height / 2,
    ]
)
cbar = plt.colorbar(im, cax=cbar_ax)
cbar.set_ticks(np.log([0.05, 0.2, 1, 6.4]))
cbar.set_ticklabels(["<5", 20, 100, ">640"])
cbar.ax.tick_params(labelsize=fontsize_dict["tick"], pad=2)
cbar.ax.set_title("Preferred\nvirtual\ndepth (cm)", fontsize=fontsize_dict["tick"], ha="left")

# Plot gradient quantification
ax = fig.add_axes([0.25, 0.45, 0.15, 0.15], polar=True)
ax.hist(
    np.arctan2(this_coef[:, 1], this_coef[:, 0]),
    bins=50,
    color="black",
    edgecolor="black",
)
ax.set_xticks(np.linspace(0, 2 * np.pi, 4, endpoint=False))
ax.set_xticklabels(["Temporal", "Upper", "Nasal", "Lower"], fontsize=fontsize_dict["tick"])
ax.set_yticks([])
ax.tick_params(axis="x", pad=1)
print(f"P-value for gradient for uncorrected depth: {ztest_2d(coefs)[0]}")
print(f"P-value for gradient for corrected depth: {ztest_2d(coefs_corrected)[0]}")

# Plot scatter map for each depth range
for i, (r, text) in enumerate(zip(ranges, ["<20 cm", "20-100 cm", ">100 cm"])):
    ax = fig.add_axes([0.5 + 0.15 * i, 0.35, 0.1, 0.15])
    idx = (neurons_df_sig[col].values > r[0]) & (neurons_df_sig[col].values <= r[1])
    print(f"number of neurons {len(neurons_df_sig[idx])}")
    ax = sns.scatterplot(
        neurons_df_sig[idx],
        x="rf_azi_jittered",
        y="rf_ele_jittered",
        hue=np.log(neurons_df_sig[idx][col]),
        palette="cool_r",
        s=1,
        alpha=0.2,
        linewidth=0,
        legend=False,
        ax=ax,
        hue_norm=norm,
        rasterized=True,
    )
    if i == 0:
        plt.ylabel("RF elevation (degrees)", fontsize=fontsize_dict["label"])
    else:
        plt.ylabel("")
    plt.xlabel("")
    ax.set_xlim([15, 120])
    ax.set_ylim([-40, 40])
    format_plot(ax)

    ax.text(120, 45, text, ha="right", fontsize=fontsize_dict["label"])
    zz = calculate_zz(idx)
    ax = fig.add_axes([0.5 + 0.15 * i, 0.1, 0.1, 0.15])
    im = plt.imshow(
        zz,
        extent=im_extent,
        origin="lower",
        cmap="Reds",
        vmin=0,
        vmax=0.6,
    )
    format_plot(ax)
    if i == 1:
        plt.xlabel("RF azimuth (degrees)", fontsize=fontsize_dict["label"])
    if i == 2:
        ax_pos = ax.get_position()
        cbar_ax = plt.gcf().add_axes(
            [
                ax_pos.x0 + ax_pos.width * 1.1,
                ax_pos.y0,
                0.005,
                ax_pos.height / 2,
            ]
        )
        cbar = plt.colorbar(im, cax=cbar_ax, ticks=[0, 0.6])
        cbar.ax.tick_params(labelsize=fontsize_dict["tick"], pad=2)

plt.savefig(SAVE_ROOT / f"rf_position_{mode}.svg", bbox_inches="tight", dpi=300)

In [ ]:
# Plot RF location correction
xrange = 27.5, 117.5
yrange = -37.5, 37.5
xx, yy = np.mgrid[xrange[0] : xrange[1] + 5 : 5, yrange[0] : yrange[1] + 5 : 5]
im_extent = (xrange[0] - 2.5, xrange[1] + 2.5, yrange[0] - 2.5, yrange[1] + 2.5)

correction = np.log2(
    1
    / np.sqrt(
        (np.sin(np.deg2rad(xx)) ** 2) * (np.cos(np.deg2rad(yy)) ** 2)
        + (np.sin(np.deg2rad(yy)) ** 2)
    )
)

plt.imshow(correction.T, extent=im_extent, origin="lower")
plt.colorbar()

## Stats

In [ ]:
# Calculate the p values for correlation between overview position and preferred depth
plt.figure(figsize=(15, 4))
for i, col in enumerate(corrs):
    plt.subplot(1, 5, i + 1)
    plt.hist(corrs[col], bins=20, color="k")
    # plot a line at 0.025 and 0.975 quantiles
    plt.axvline(np.quantile(corrs[col], 0.025), c="r")
    plt.axvline(np.quantile(corrs[col], 0.975), c="r")
    # find proportion of values on the other side of 0 from the median
    p = np.mean(np.array(corrs[col]) > 0)
    if p > 0.5:
        p = 1 - p
    p *= 2
    print(f"{col}: p = {p}")
    plt.title(f"{col} p = {p:.3f}")